# Holiday Destination Recommender - Report

## 1. Introduction

I am sure you know this problem very well.

You are so ready to go onto your next vacation - only you do not know what destination you should choose. Not so much because you have too much choice and you cannot decide - the issue is rather, you do not actually know what are the destinations that *really* match your current *mood* for holiday destinations? And is there anything new and exotic to discover other than all the mainstream places?

So here is the idea.

A *Holiday Destination Recommender*: You tell the recommender the places you like and that match your current mood & the recommender shows you similar places worth visting - places you might have never come across yourself.
Yes, very similar to what you already know from Netflix, Spotify and YouTube.

Imagine this feature on the airline websites.

The project is implemented using an unsupervised learning approach and hence does not depend on the availability of labeled training data. Please refer to section **Methodology** for more details on the approach.



## 2. Data

In this section, I present which type of data is needed, where they are retrieved from and how they are processed to implement the Holiday Destination Recommender.

## 2.1 Data Requirements

To implement the Holiday Destination Recommender, the following data is required:
- **A list of cities that serve as "benchmark cities"**; in this example, we go with the cities "Barcelona, Spain" and "Budapest, Hungary"
- **A list of all cities in the world**: This is needed, so that the recommender can scout new cities to suggest to the user.
- **The latitudes and longitudes of those cities**: This is needed, so that places are uniquely identified and more information can be retrieved about them.
- **The available venues of those cities**: This is needed, so that the recommender can calculate the similarity between the benchmark cities and possible holiday destinations. First and foremost, the available *venue categories* are of interest, since they form the basis of the comparison.

## 2.2 Data Sources

The following data sources are used to retrieve above mentioned data:
- Dataset from https://datahub.io/core/world-cities: Here a CSV is provided that contains all major cities of the world with more than 150000 inhabitants. In total there are more than 20k entries.
- Python library *geopy* to request *Nominatim* interface for retrieving the coordinates of cities.
- Foursquare Places API to obtain information on the available venues for a city.

## 2.3 Data Processing and Preparation

This section depicts the applied data processing flow.


#### 2.3.1 Data Retrieval

1. The user provides a list of benchmark cities as input to the recommender. 
2. The coordinates are retrieved using *geopy* library to request *Nominatim* API.
3. The venues are queried from Foursquare Places API and listed along with their venue category.
4. Above process is repeated for the destination cities, which are read from the CSV provided by https://datahub.io/core/world-cities. A random sample of 100 cities is taken from that list: The limit of 100 cities is in order not to exceed the daily requests limit imposed by the Foursquare Places API on the Sandbox account; the random selection makes sure the recommender always explores new cities.

Below you find an extract of the benchmark cities with coordinates and available venues.

![title](img/Benchmark_Cities.png)

And the same for the destination cities.

![title](img/New_Cities.png)

#### 2.3.2 Data Preparation

1. The retrieved dataframes are turned into a one-hot encoding representation of cities vs. venue categories.
2. The frequency per venue category and place (i.e. the ratio between the number of a particular venue category appearing vs. the total number of venues for a city) is calculated based on the one-hot encoding.

Below you find an extract of a) the one-hot encoding and b) the calculated venue frequencies for the benchmark cities.

![title](img/Benchmark_Cities_Onehot.png)

![title](img/Benchmark_Cities_Freq.png)

And the same for the destination cities.

![title](img/New_Cities_Onehot.png)

![title](img/New_Cities_Freq.png)

In the next section, I will discuss how the calculated frequencies of the venue categories are used to generate destination recommendations.

## 3. Methodology

### 3.1 Basic Idea

In order to avoid depending on large amount of high-quality labeled training data, an unsupervised approach is followed. 

A very common technique to implement unsupervised recommender systems is to use *Association Rules*: For the Holiday Destination Recommender, *Association Rule Learning* could be applied by analyzing the relations between the places a particular user has liked. By replicating this analysis for a significantly large amount of users, rules can be derived to extrapolate for a given set of benchmark cities, what are other cities that have high likelihood to be endorsed as well. 

Unfortunately, the Foursquare Places API does not expose the information which places a user has liked (unless the user is a "friend").

**So I have to pivot for an alternative unsupervised approach**: 
1. **Calculating similarity between cities**: Based on the available venue categories of a city, similarity scores are calculated between each potential destination city and the benchmark cities.
2. **Clustering potential destination cities**: Based on the calculated similarity scores, a clustering algorithm is applied in order to identify the destination cities worth paying a visit. This step would not be necessary if potential destination cities are only compared to one single benchmark city; in that case, manual clustering into e.g. "similar" or "non-similar" destination cities would be easily possible. However, since the Holiday Destination Recommender is supposed to work with multiple benchmark cities, manually identifying the right number of clusters and correctly assigning a large amount of destination cities to the right cluster is inefficient. This will become more clear when we look at the insights in the **Results** section.

### 3.2 Calculating the Similarity Scores

In order to calculate the similarity between potential destination cities and the benchmark cities, the measure of Cosine Similarity is used.

Cosine Similarity is preferred in this case over other distance calculating methods (such as Euclidiean, Levenshtein, Hamming etc.) since it has proven effective in similar use cases such as in calculating the similarity of texts: In the context of text comparison, Cosine Similarity is used to measure the similarity of texts based on the *frequencies* of the terms appearing in the texts. This is comparable to the case of the Holiday Destination Recommender, where similarities between potential destination cities and benchmark cities are to be calculated based on the *frequency* of the available venue categories.

The Cosine Similarity scores are calculated using the built-in function from the Python library *Scikit-Learn*. The output is an array matrix, in this case with the column labels being the benchmark cities and the row labels being the potential destination cities; their intersections represent the similarity between a potential destination city and the benchmark city.

Below you can find an extract of the calculated Cosine Similarity scores:

![title](img/Similarity_Scores.png)

### 3.3 Calculating the Optimal Number of Clusters

Before running the *k-means clustering* algorithm on the calculated similarities, the optimal number of clusters has to be identified. Various methods exist to determine the optimal *k*, a simple yet reliable method is the *Silhouette method*: For a range of values for *k*, a *Silhouette score* is calculated based the resulting clusters; *k* is optimal when the *Silhouette score* is peaking. 

The Silhouette scores are calculated using the built-in function from the Python library *Scikit-Learn*.

In our example, the optimal number of clusters is *k = 5*:

![title](img/Silhouette.png)

### 3.4 K-Means Clustering

At last, everything is ready for the *K-Means* clustering algorithm:

The potential destination cities are clustered with *k = 5* based on their Cosine Similarity scores, that have been calculated according to the frequencies of the available venue categories.

The *K-Means Clustering* algorithm is run using the built-in function from the Python library *Scikit-Learn*.

## 4. Results

The *K-Means Clustering* algorithm has yielded 5 clusters with the following distribution of the potential destination cities:

![title](img/Cluster_Statistics.png)

On the world map the destination cities are distributed as follows, with different clusters having different colors:

![title](img/Map.png)

In order to better understand how the different clusters distinguish from each other, the average of the similarity scores to each of the benchmark cities is calculated with the following results:

![title](img/Cluster_Means.png)

In the next section, the interpretation of these data insights will be discussed.

## 5. Discussion

Based on the average similarity scores for each cluster shown above, it can be seen that the following cities are worth a visit:
- Cities very similar to Barcelona but very dissimilar to Budapest from **Cluster 4**
- Cities very similar to Budapest but dissimilar to Barcelona from **Cluster 1**
- Cities similar to Budapest and similar to Barcelona from **Cluster 3**

Cities from cluster 2 definitely should **not** be visited as they are neither similar to Barcelona nor to Budapest.

Below you find the cities from **Cluster 4**, *cities very similar to Barcelona but very dissimilar to Budapest*:

![title](img/Cluster4.png)

Below you find the cities from **Cluster 1**, *cities very similar to Budapest but dissimilar to Barcelona*:

![title](img/Cluster1.png)

Below you find the cities from **Cluster 3**, *cities similar to Budapest and similar to Barcelona*:

![title](img/Cluster3.png)

Below you find the cities from **Cluster 2**, *cities neither similar to Barcelona nor to Budapest*:

![title](img/Cluster2.png)

## 6. Conclusion

Using different data sources like Foursquare Places API, libraries like *Scikit-Learn* and machine learning algorithms like *K-Means Clustering* it was possible to build a simple Holiday Destination Recommender - a recommender that suggests new places in the world to visit that the user would have never come across otherwise.

For future versions, the following areas can be further improved to achieve even better and more accurate recommendations:

### Feature Engineering

Currently, only venue categories are considered for the comparison. However, **additional features** can be considered for the comparison, such as: **population, area size** etc.


Furthermore, at the moment *all* venue categories of the benchmark cities are considered for the similarity comparison, also those where the frequency is low. In other words, two cities might also be considered similar, because they are *short* of the same venue categories.
Giving it some more thoughts, it would make sense to make sure that two cities are **only considered similar if they share the same *available* venue categories. A user is *not interested* in visiting a place because of the *absence* of certain venue categories.**

### Feature Transformation

In the current version, the frequency of the venue categories are used for calculating the similarity scores. However, users may not only be interested how often a venue category is available in a city compared to the total number of venues in a city. It is fair to assume, that **the actual number of venues in a particular category is also of interest. In order to mitigate this, the total number of venues per city should also be considered during the comparison.**

### Hyperparameter Tuning

At the moment, the *K-Means Clustering* algorithm is run only with the default parameters. For future versions, **hyperparameter tuning (e.g. number of iterations, number of initializations)** should be attempted for better clustering results.

### Sample Size

Due to the number of requests per day being limited to 950 (as of 30th March 2020) by the Foursquare Places API, a sample of only 100 potential destination cities is taken from the list of all cities. In order to **increase the probability of finding highly similar potential destination cities, the Foursquare account should be graded and a larger sample size should be chosen.**

### Post-Processing

As of now, all clustered potential destination cities are presented to the users. In order to guarantee the quality of suggested holiday destinations, it is thinkable that in a post-processing step **only potential destination cities with a similarity score higher than a certain threshold** are filtered and recommended.